In [7]:
import torch
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
)
from peft import  PeftModel  

In [8]:
def test_ppo_vs_base(prompt, model_name, lora_path="./qwen2.5-1.5b-ppo-lora"):
    # 加载基础模型
    base_model = AutoModelForCausalLM.from_pretrained(
        model_name,
        torch_dtype=torch.bfloat16 if torch.cuda.is_available() else torch.float32,
        device_map="auto",
    )
    base_model.eval()  # 基础模型设为评估模式
    
    # 加载tokenizer（保持与原逻辑一致，使用lora路径的tokenizer）
    tokenizer = AutoTokenizer.from_pretrained(lora_path)
    tokenizer.pad_token = tokenizer.eos_token
    
    # 处理输入
    messages = [{"role": "user", "content": prompt}]
    text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    inputs = tokenizer(text, return_tensors="pt").to(base_model.device)
    
    # 基础模型生成
    with torch.no_grad():
        base_outputs = base_model.generate(
            **inputs,
            max_new_tokens=100,
            do_sample=True,
            top_k=50,
            top_p=0.9,
            temperature=0.7,
            pad_token_id=tokenizer.eos_token_id
        )
    base_generated = tokenizer.decode(base_outputs[0], skip_special_tokens=True)
    base_result = base_generated.split("assistant\n")[-1].strip()
    
    # 加载PPO模型（base + LoRA）
    ppo_model = PeftModel.from_pretrained(base_model, lora_path)  # 加载LoRA适配器
    ppo_model.eval()
    
    # PPO模型生成
    with torch.no_grad():
        ppo_outputs = ppo_model.generate(
            **inputs,
            max_new_tokens=100,
            do_sample=True,
            top_k=50,
            top_p=0.9,
            temperature=0.7,
            pad_token_id=tokenizer.eos_token_id
        )
    ppo_generated = tokenizer.decode(ppo_outputs[0], skip_special_tokens=True)
    ppo_result = ppo_generated.split("assistant\n")[-1].strip()
    
    return base_result, ppo_result

# 示例测试
test_prompt = "Python中的字典是什么？"
base_result, ppo_result = test_ppo_vs_base(test_prompt, model_name="/root/All_in_llm/models/Qwen2.5-1.5B-Instruct")  
print(f"基础模型生成: {base_result}")
print(f"PPO模型生成: {ppo_result}")

基础模型生成: 在Python中，字典是一种无序的、可变的数据结构，它存储一组键值对。每个键（key）都有一个对应的值（value），并且键必须是唯一的。字典使用一对花括号{}来表示，并且可以包含任意数量和类型的键值对。

以下是一些基本概念：

1. 键：字典的唯一标识符，可以是任何不可变类型，如字符串、数字或元组。

2. �
PPO模型生成: 在Python中，字典是一种数据结构，它用于存储键值对。每个键都是唯一的，并且可以是任何可哈希的类型（例如字符串、数字或列表），而对应的值也可以是任何类型的对象。

字典通常用大括号 `{}` 来表示，其中包含一对对键和值的元素。这些元素之间使用冒号 `:` 分隔，然后在同一行上用逗号 `,` 隔开其他元素。

以下是一
